# ☕ Coffee Sales Analysis Project


#  Table of Contents

1. [Project Title & Team](#project-title-coffee-shop-sales-analysis)
2. [Problem Statement](#problem-statement)
3. [Objectives](#objectives)
4. [Dataset Overview](#-dataset-overview)
    - [Feature Description](#-feature-description)
    - [Dataset Link](#-dataset-link)
5. [Methodology](#methodology)
    - Data Cleaning
    - Exploratory Data Analysis (EDA)
    - Modeling & Evaluation
6. [Data Cleaning](#data-cleaning)
7. [Exploratory Data Analysis (EDA)](#eda)
    - Sales by Weekday
    - Coffee Popularity
    - Average Spending by Time of Day
    - Weekday vs Weekend Sales
    - Monthly & Seasonal Trends
    - Best Coffee Type in March
    - Best Coffee Type by Season
    - Sales by Hour of Day
    - Yearly/Monthly Comparisons
    - Correlation Heatmap
8. [Modeling](#model)
    - OLS Regression
    - Linear Regression
    - Random Forest & XGBoost
    - Model Comparison
9. [Insights from EDA](#-insights-from-eda)
10. [Business Recommendations](#business-recommendations)

# Table of Contents

# Project Title: Coffee Shop Sales Analysis

## Team Members & Roles
- Mudhawi Saleh Alshiha : Data Cleaning & EDA & answer quetions  
- Musab Alabdullatif : EDA & answer quetions & modeling   
- Mousa Hamoud : Modeling & Evaluation &  Dashboard




# Problem Statement  

Coffee shops operate in highly competitive environments where customer preferences, peak hours, and seasonal demand directly influence revenue.  
Without proper analysis, businesses may miss opportunities to optimize staffing, adjust pricing strategies, or promote the right products at the right time.  

This project aims to analyze coffee shop transaction data to uncover patterns in sales, payment methods, and customer behavior.  
By applying statistical models and predictive analytics, we can provide actionable recommendations that support better decision-making and improved business performance.


# Objectives

We will answer the following questions:  
- On which day of the week are there the most sales?

- What is the most popular coffee type by number of orders?

- What is the average money spent in the morning vs afternoon vs evening?

- Are weekends vs weekdays different in average sales?

- Are there seasonal or monthly trends in coffee sales, and which months generate the highest revenue?  

- Which hour of the day generates the most sales?

- and more
---------------------------------------------------
- Build and evaluate statistical & ML models for sales prediction.

- Provide actionable business recommendations.  


##  Dataset Overview

This dataset contains **coffee shop transaction records** across **2024 and early 2025**.  
It captures details about each purchase such as the **time of sale, coffee type, payment method, transaction value, and temporal attributes** (weekday, month, season).  

The dataset is well-suited for **business analysis, trend detection, customer behavior insights, and predictive modeling**.

---

###  Feature Description

| Column        | Description                                                                 |
|---------------|-----------------------------------------------------------------------------|
| `Date`        | Full date of the transaction                                                |
| `hour_of_day` | Hour when the purchase was made (0–23)                                      |
| `cash_type`   | Mode of payment (*cash* / *card*)                                           |
| `money`       | Transaction amount (local currency)                                         |
| `coffee_name` | Type of coffee purchased (Latte, Americano, Cappuccino, etc.)               |
| `Time_of_Day` | Categorized time of purchase (*Morning, Afternoon, Night*)                  |
| `Weekday`     | Day of the week (Mon–Sun)                                                   |
| `Month_name`  | Month of the purchase (Jan–Dec)                                             |
| `Weekdaysort` | Numeric representation for weekday ordering (1 = Monday, 7 = Sunday)        |
| `Monthsort`   | Numeric representation for month ordering (1 = January, 12 = December)      |
| `Season`      | Seasonal grouping of transactions (Winter, Spring, Summer, Autumn)          |

---

### 🔗 Dataset Link  
[Coffee Sales Dataset](https://www.kaggle.com/datasets/navjotkaushal/coffee-sales-dataset/data)


# Methodology
will contains:
- Data cleaning
- EDA
- MODEL

In [1]:
# import necessary libraries
import pandas as pd
import numpy as np
import plotly.express as px
import statsmodels.api as sm


## DATA Cleaning

In [3]:
# load data
df = pd.read_csv('Coffe_sales.csv')

In [4]:
# Display basic information about the dataset
print("Dataset Shape:", df.shape)
print("\nColumn Names:")
print(df.columns.tolist())
print("\nFirst 5 rows:")
df.head()

Dataset Shape: (3547, 11)

Column Names:
['hour_of_day', 'cash_type', 'money', 'coffee_name', 'Time_of_Day', 'Weekday', 'Month_name', 'Weekdaysort', 'Monthsort', 'Date', 'Time']

First 5 rows:


,hour_of_day,cash_type,money,coffee_name,Time_of_Day,Weekday,Month_name,Weekdaysort,Monthsort,Date,Time
0,10,card,38.7,Latte,Morning,Fri,Mar,5,3,2024-03-01,10:15:50.520000
1,12,card,38.7,Hot Chocolate,Afternoon,Fri,Mar,5,3,2024-03-01,12:19:22.539000
2,12,card,38.7,Hot Chocolate,Afternoon,Fri,Mar,5,3,2024-03-01,12:20:18.089000
3,13,card,28.9,Americano,Afternoon,Fri,Mar,5,3,2024-03-01,13:46:33.006000
4,13,card,38.7,Latte,Afternoon,Fri,Mar,5,3,2024-03-01,13:48:14.626000


In [5]:
# check null values
df.isnull().sum()


,0
hour_of_day,0
cash_type,0
money,0
coffee_name,0
Time_of_Day,0
Weekday,0
Month_name,0
Weekdaysort,0
Monthsort,0
Date,0


In [6]:
# check duplicated values
df.duplicated().sum()

np.int64(0)

In [7]:
# Check for outliers in 'money' using Plotly box plot

fig = px.box(df, y='money', points='all', title='Outlier Detection for Sales (money)')
fig.show()

## EDA

In [8]:
# see distribution of year from column 'Date'
df['Date'] = pd.to_datetime(df['Date'])
df['year'] = df['Date'].dt.year
df['year'].value_counts()


,count
year,
2024,2604
2025,943


In [9]:
# Make sure Date is in datetime format
df['Date'] = pd.to_datetime(df['Date'])

# Extract year and month
df['year'] = df['Date'].dt.year
df['month'] = df['Date'].dt.month

# Find the maximum year and month in dataset
last_year = df['year'].max()
last_month = df[df['year'] == last_year]['month'].max()

print("Dataset stops at:")
print("Year:", last_year)
print("Month:", last_month)
#  last transaction date
last_date = df['Date'].max()
print("Last transaction date in dataset:", last_date.date())


Dataset stops at:
Year: 2025
Month: 3
Last transaction date in dataset: 2025-03-23


### 1-Which day of the week has the most total sales?

In [10]:
# Definding the numpy arrays
money         = df["money"].to_numpy()
weekday_str   = df["Weekday"].to_numpy()
weekday_sort  = df["Weekdaysort"].to_numpy()   # numeric order for days
coffee        = df["coffee_name"].to_numpy()
time_of_day   = df["Time_of_Day"].to_numpy()
hour  = df["hour_of_day"].to_numpy()
year  = df["year"].to_numpy()

In [11]:
# 1. Group by Weekdaysort for all data
# Starting from Monday, since the dataset considers Monday as the first day of the week.
sort_keys, inv_weekdays = np.unique(weekday_sort, return_inverse=True)
print(sort_keys, inv_weekdays)
total_sales_by_sort = np.zeros(sort_keys.shape[0], dtype=float)
for i, amt in enumerate(money):
    total_sales_by_sort[inv_weekdays[i]] += amt

# 2. pick a representative weekday label for each sort key (first occurrence)
labels_by_sort = np.array([weekday_str[weekday_sort == k][0] for k in sort_keys])
print("days sorted: ", labels_by_sort)
print("total sales sorted from the max till min: ", total_sales_by_sort)

# 3. finding the result
max_idx = int(np.argmax(total_sales_by_sort))
day_with_most_sales = labels_by_sort[max_idx]
total_on_that_day = float(total_sales_by_sort[max_idx])

print("Day with most total sales:", day_with_most_sales)
print("Total sales on that day:", round(total_on_that_day, 2))

# Visualization Using Plotly
fig = px.bar(
    x=labels_by_sort,
    y=total_sales_by_sort,
    labels={"x": "Weekday", "y": "Total Sales"},
    title="Total Sales by Weekday (All Years)"
)
fig.update_layout(xaxis_title="Weekday", yaxis_title="Total Sales")
fig.show()





[1 2 3 4 5 6 7] [4 4 4 ... 6 6 6]
days sorted:  ['Mon' 'Tue' 'Wed' 'Thu' 'Fri' 'Sat' 'Sun']
total sales sorted from the max till min:  [17363.1  18168.38 15750.46 16091.4  16802.66 14733.52 13336.06]
Day with most total sales: Tue
Total sales on that day: 18168.38


### 2- What is the most popular coffee type by number of orders?



In [12]:
# 1. Declearing the variables
fav_coffee, coffee_counts = np.unique(coffee, return_counts=True)
# 2. Finding the most ordered coffe by name and count
top_idx = int(np.argmax(coffee_counts))
most_popular_coffee = fav_coffee[top_idx]
most_popular_count  = int(coffee_counts[top_idx])

print("Most popular coffee:", most_popular_coffee)
print("Number of orders:", most_popular_count)

# 2. Sorting to start with largest number (descending)
order = np.argsort(coffee_counts)[::-1]
coffee_sorted = fav_coffee[order]
counts_sorted = coffee_counts[order]

# 3. Use a loop to neatly display each coffee type with its order count
for coffee, count in zip(coffee_sorted, counts_sorted):
    print(f"{coffee}: {count}")


# Visualization Using Plotly
fig = px.bar(
    x=coffee_sorted,
    y=counts_sorted,
    labels={"x": "Coffee Type", "y": "Number of Orders"},
    title="Coffee Popularity (Order Count)"
)
fig.update_layout(xaxis_title="Coffee Type", yaxis_title="Orders")
fig.show()


Most popular coffee: Americano with Milk
Number of orders: 809
Americano with Milk: 809
Latte: 757
Americano: 564
Cappuccino: 486
Cortado: 287
Hot Chocolate: 276
Cocoa: 239
Espresso: 129


### 3- What is the average money spent in the morning vs afternoon vs evening?

In [13]:
# 1. Declaring the variables
period_of_time = np.array(["Morning", "Afternoon", "Night"])

# Get arrays directly from the DataFrame to keep them aligned
time_of_day = df["Time_of_Day"].values
money = df["money"].values
present = np.unique(time_of_day)

# 2. Finding the averages per period of the day
ordered_times = np.array(
    [t for t in period_of_time if t in present] + [t for t in present if t not in period_of_time],
    dtype=object
)

avg_spend = np.zeros(len(ordered_times), dtype=float)
for i, t in enumerate(ordered_times):
    mask = (time_of_day == t)
    avg_spend[i] = np.mean(money[mask]) if np.any(mask) else np.nan

# Print results
print("Average spending by time of day:")
for t, a in zip(ordered_times, avg_spend):
    print(f"{t}: {np.round(a, 2)}")

# Visualization Using Plotly
fig = px.bar(
    x=ordered_times,
    y=avg_spend,
    labels={"x": "Time of Day", "y": "Average Money Spent"},
    title="Average Spending by Time of Day"
)
fig.update_layout(xaxis_title="Time of Day", yaxis_title="Average Spend")
fig.show()



Average spending by time of day:
Morning: 30.42
Afternoon: 31.64
Night: 32.89


### 4-Are weekends vs weekdays different in average sales?

In [14]:
# 1. Create a new column for weekend flag
df["is_weekend"] = df["Weekdaysort"].apply(lambda x: "Weekend" if x >= 5 else "Weekday")

# 2. Calculate average spending for weekdays vs weekends
avg_sales = df.groupby("is_weekend")["money"].mean().reset_index()

print("\n=== Average Sales: Weekday vs Weekend ===")
for row in avg_sales.itertuples(index=False):
    print(f"{row.is_weekend}: {round(row.money, 2)}")

# 3. Visualization
fig = px.pie(
    avg_sales,
    names="is_weekend",
    values="money",
    color="is_weekend",
    color_discrete_map={"Weekday": "skyblue", "Weekend": "orange"},
    title="Average Sales: Weekdays vs Weekends"
)

fig.update_traces(textposition="inside", textinfo="percent+label")
fig.show()



=== Average Sales: Weekday vs Weekend ===
Weekday: 31.69
Weekend: 31.58


### 5- Are there seasonal or monthly trends in coffee sales, and which months generate the highest revenue?  

In [15]:
# 1. Group by month
monthly_sales = df.groupby(["Monthsort", "Month_name"])["money"].sum().reset_index()

# 2. Find the month with the highest revenue
best_idx = monthly_sales["money"].idxmax()
best_month = monthly_sales.loc[best_idx, "Month_name"]
best_value = monthly_sales.loc[best_idx, "money"]

print("\n=== Monthly Sales Analysis ===")
print("Month with highest total sales:", best_month, "→", round(best_value, 2))

# 3. Visualization
fig = px.line(
    monthly_sales,
    x="Monthsort",
    y="money",
    text="Month_name",
    markers=True,
    labels={"Monthsort": "Month", "money": "Total Sales"},
    title="Monthly Coffee Sales Trend"
)
fig.update_traces(textposition="top center")
fig.update_layout(
    xaxis=dict(
        tickmode="array",
        tickvals=monthly_sales["Monthsort"],
        ticktext=monthly_sales["Month_name"]
    ),
    xaxis_title="Month",
    yaxis_title="Total Sales"
)
fig.show()

# 4. Visualization Using Plotly
fig = px.bar(
    monthly_sales,
    x="Month_name",
    y="money",
    color="money",
    color_continuous_scale="tealgrn",
    labels={"Month_name": "Month", "money": "Total Sales"},
    title="Total Coffee Sales by Month"
)
fig.update_layout(xaxis_title="Month", yaxis_title="Total Sales")
fig.show()



# 1. Map months to seasons
season_map = {
    12: "Winter", 1: "Winter", 2: "Winter",
    3: "Spring", 4: "Spring", 5: "Spring",
    6: "Summer", 7: "Summer", 8: "Summer",
    9: "Autumn", 10: "Autumn", 11: "Autumn"
}
df["Season"] = df["Monthsort"].map(season_map)

# 2. Group sales by season
seasonal_sales = df.groupby("Season")["money"].sum().reset_index()

# 3. Find the season with the highest revenue
best_idx = seasonal_sales["money"].idxmax()
best_season = seasonal_sales.loc[best_idx, "Season"]
best_value = seasonal_sales.loc[best_idx, "money"]

print("\n=== Seasonal Sales Analysis ===")
print("Season with highest total sales:", best_season, "→", round(best_value, 2))

# 4. Visualization
fig = px.bar(
    seasonal_sales,
    x="Season",
    y="money",
    color="money",
    color_continuous_scale="viridis",
    labels={"Season": "Season", "money": "Total Sales"},
    title="Total Coffee Sales by Season"
)
fig.update_layout(xaxis_title="Season", yaxis_title="Total Sales")
fig.show()

# 5. Visualization
fig = px.pie(
    seasonal_sales,
    names="Season",
    values="money",
    color="Season",
    color_discrete_map={"Winter": "lightblue", "Spring": "green", "Summer": "orange", "Autumn": "brown"},
    title="Seasonal Share of Coffee Sales"
)
fig.update_traces(textposition="inside", textinfo="percent+label")
fig.show()




=== Monthly Sales Analysis ===
Month with highest total sales: Mar → 15891.64



=== Seasonal Sales Analysis ===
Season with highest total sales: Autumn → 32470.34


### 6- Best Coffee Type in March (highest month)

In [16]:
# 1. Filter March
df_march = df[df["Month_name"] == "Mar"]

# 2. Group sales by coffee type
march_sales = df_march.groupby("coffee_name")["money"].sum().reset_index()

# 3. Find top coffee type
best_idx = march_sales["money"].idxmax()
best_type = march_sales.loc[best_idx, "coffee_name"]
best_value = march_sales.loc[best_idx, "money"]

print("\n=== Best Coffee in March ===")
print("Coffee type with highest total sales:", best_type, "→", round(best_value, 2))

# 4. Visualization
fig = px.bar(
    march_sales,
    x="coffee_name",
    y="money",
    color="money",
    color_continuous_scale="tealgrn",
    labels={"coffee_name": "Coffee Type", "money": "Total Sales"},
    title="Coffee Sales in March"
)
fig.update_layout(xaxis_title="Coffee Type", yaxis_title="Total Sales")
fig.show()



=== Best Coffee in March ===
Coffee type with highest total sales: Americano → 3572.72


### 7- Best Coffee Type in Each Season

In [17]:
# 1. Group sales by Season + Coffee Type
season_coffee_sales = df.groupby(["Season", "coffee_name"])["money"].sum().reset_index()

# 2. For each season, find the top coffee
best_coffee_per_season = season_coffee_sales.loc[season_coffee_sales.groupby("Season")["money"].idxmax()]

print("\n=== Best Coffee Type by Season ===")
for row in best_coffee_per_season.itertuples(index=False):
    print(f"{row.Season}: {row.coffee_name} → {round(row.money, 2)}")

# 3. Visualization
fig = px.bar(
    season_coffee_sales,
    x="Season",
    y="money",
    color="coffee_name",
    barmode="group",
    labels={"Season": "Season", "money": "Total Sales"},
    title="Coffee Sales by Type Across Seasons"
)
fig.update_layout(xaxis_title="Season", yaxis_title="Total Sales")
fig.show()



=== Best Coffee Type by Season ===
Autumn: Latte → 9837.36
Spring: Latte → 6312.94
Summer: Americano with Milk → 6040.16
Winter: Americano with Milk → 5986.84


### 8- Which hour of the day generates the most sales?

In [18]:
# 1. Declaring the variable and index
hour = df["hour_of_day"].values
money = df["money"].values
hours, inv_hour = np.unique(hour, return_inverse=True)

# 2. Finding the total sales per hour
totals_by_hour = np.bincount(inv_hour, weights=money).astype(float)

# 3. find top hour with most sales
best_idx = np.argmax(totals_by_hour)
best_hour = hours[best_idx]
best_value = totals_by_hour[best_idx]

print("\n=== All Years ===")
print("Hour with highest total sales:", best_hour, "→", round(best_value, 2))

# Visualization
fig = px.line(
    x=hours,
    y=totals_by_hour,
    markers=True,
    labels={"x": "Hour of Day", "y": "Total Sales"},
    title="Total Sales by Hour of Day (All Years)"
)
fig.update_layout(xaxis=dict(dtick=1))  #
fig.show()


=== All Years ===
Hour with highest total sales: 10 → 10198.52


### Correlation Heatmap

In [19]:
import plotly.express as px

# Select numeric columns for correlation
numeric_cols = ["hour_of_day", "Weekdaysort", "Monthsort", "money"]
corr = df[numeric_cols].corr()

# Plotly heatmap
fig = px.imshow(
    corr,
    text_auto=True,
    color_continuous_scale="YlGnBu",
    title="Correlation Heatmap"
)
fig.show()

## Model

In [32]:
# 1. Create a proper copy of the dataframe
df_model = df.copy()

In [33]:
 #2. Drop unwanted columns
df_model.drop(['Time', 'cash_type', 'Month_name', 'Weekday'], axis=1, inplace=True)

In [34]:
# 3. Process the date column and extract feature
df_model['Date'] = pd.to_datetime(df_model['Date'], format='%Y-%m-%d')
df_model['Year'] = df_model['Date'].dt.year.astype(str)
df_model['Month'] = df_model['Date'].dt.month
df_model['Season'] = df_model['Month'].map({
    12:'Winter',1:'Winter',2:'Winter',
    3:'Spring',4:'Spring',5:'Spring',
    6:'Summer',7:'Summer',8:'Summer',
    9:'Autumn',10:'Autumn',11:'Autumn'
})

In [35]:
# Target variable (log transform to reduce skewness)
y = np.log1p(df_model["money"])

# Features
categorical_cols = ['coffee_name', 'Time_of_Day', 'Year', 'Season']
numerical_cols = ["hour_of_day", "Weekdaysort", "Monthsort"]

In [36]:
# Encoding
X_cat = pd.get_dummies(df_model[categorical_cols], drop_first=True, dtype=int)
X_num = (df_model[numerical_cols] - df_model[numerical_cols].mean()) / df_model[numerical_cols].std()
X_combined = pd.concat([X_num, X_cat], axis=1)

In [37]:
# Add constant
X_final = sm.add_constant(X_combined)

# Split
n = len(X_final)
train_size = int(0.8*n)
idx = np.random.permutation(n)
train_idx, test_idx = idx[:train_size], idx[train_size:]

X_train, X_test = X_final.iloc[train_idx], X_final.iloc[test_idx]
y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]


In [38]:
# Train with robust SE
model = sm.OLS(y_train, X_train).fit(cov_type="HC3")


In [39]:
# Predict
y_pred = model.predict(X_test)


In [40]:
# Metrics
ssr = np.sum((y_test - y_pred)**2)
sst = np.sum((y_test - np.mean(y_test))**2)
r2 = 1 - ssr/sst
rmse = np.sqrt(np.mean((y_test - y_pred)**2))


In [41]:
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                  money   R-squared:                       0.931
Model:                            OLS   Adj. R-squared:                  0.931
Method:                 Least Squares   F-statistic:                     2786.
Date:                Sat, 20 Sep 2025   Prob (F-statistic):               0.00
Time:                        14:36:24   Log-Likelihood:                 5004.9
No. Observations:                2837   AIC:                            -9976.
Df Residuals:                    2820   BIC:                            -9875.
Df Model:                          16                                         
Covariance Type:                  HC3                                         
                                      coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------------------
const     

In [43]:
import plotly.graph_objects as go

# Plot actual vs predicted sales for OLS model
fig_ols = go.Figure()
fig_ols.add_trace(go.Scatter(x=np.expm1(y_test), y=np.expm1(y_pred), mode='markers', name='OLS Predictions')) # Applying expm1 to revert log transformation
fig_ols.add_trace(go.Scatter(x=[np.expm1(y_test).min(), np.expm1(y_test).max()], y=[np.expm1(y_test).min(), np.expm1(y_test).max()],
                            mode='lines', name='Perfect Fit', line=dict(color='red', dash='dash')))
fig_ols.update_layout(title='Actual vs Predicted Sales (OLS Model)',
                      xaxis_title='Actual Sales (money)',
                      yaxis_title='Predicted Sales (money)')
fig_ols.show()

In [46]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_absolute_error

# Prepare data
X = df[["hour_of_day", "cash_type", "coffee_name", "Time_of_Day",
        "Weekdaysort", "Monthsort"]]
y = df["money"]
X_encoded = pd.get_dummies(X, drop_first=True)
X_encoded = X_encoded.astype(float)
# Split data
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, test_size=0.2, random_state=42)
model = LinearRegression()
model.fit(X_train, y_train)

# Make predictions
y_pred_lr = model.predict(X_test)

# Calculate R2 and MAE
r2_lr = r2_score(y_test, y_pred_lr)
mae_lr = mean_absolute_error(y_test, y_pred_lr)

print(f"Linear Regression Model R²: {r2_lr:.4f}")
print(f"Linear Regression Model MAE: {mae_lr:.4f}")

Linear Regression Model R²: 0.8714
Linear Regression Model MAE: 1.4949


In [47]:
import plotly.graph_objects as go

# Linear Regression predictions
y_pred_lr = model.predict(X_test)

# Plot actual vs predicted sales for Linear Regression model
fig_lr = go.Figure()
fig_lr.add_trace(go.Scatter(x=y_test, y=y_pred_lr, mode='markers', name='Linear Regression Predictions'))
fig_lr.add_trace(go.Scatter(x=[y_test.min(), y_test.max()], y=[y_test.min(), y_test.max()],
                            mode='lines', name='Perfect Fit', line=dict(color='red', dash='dash')))
fig_lr.update_layout(title='Actual vs Predicted Sales (Linear Regression Model)',
                      xaxis_title='Actual Sales (money)',
                      yaxis_title='Predicted Sales (money)')
fig_lr.show()

In [49]:
import xgboost as xgb
from sklearn.metrics import r2_score, mean_absolute_error
import plotly.graph_objects as go

# Initialize and train the XGBoost Regressor model
xgboost_model = xgb.XGBRegressor(objective='reg:squarederror', random_state=42)
xgboost_model.fit(X_train, y_train)

# Make predictions
y_pred_xgboost = xgboost_model.predict(X_test)

# Calculate R2 and MAE
r2_xgboost = r2_score(y_test, y_pred_xgboost)
mae_xgboost = mean_absolute_error(y_test, y_pred_xgboost)

print(f"XGBoost Model R2: {r2_xgboost:.4f}")
print(f"XGBoost Model MAE: {mae_xgboost:.4f}")

# Plot actual vs predicted sales for XGBoost model
fig_xgboost = go.Figure()
fig_xgboost.add_trace(go.Scatter(x=y_test, y=y_pred_xgboost, mode='markers', name='XGBoost Predictions'))
fig_xgboost.add_trace(go.Scatter(x=[y_test.min(), y_test.max()], y=[y_test.min(), y_test.max()],
                            mode='lines', name='Perfect Fit', line=dict(color='red', dash='dash')))
fig_xgboost.update_layout(title='Actual vs Predicted Sales (XGBoost Model)',
                      xaxis_title='Actual Sales (money)',
                      yaxis_title='Predicted Sales (money)')
fig_xgboost.show()

XGBoost Model R2: 0.9758
XGBoost Model MAE: 0.3641


## Insights from EDA

### 1. Sales by Weekday
- **All Years Combined:** Sales peak on **Monday and Tuesday**, with weekends (Saturday and Sunday) showing lower totals.  
---

### 2. Coffee Popularity
- **Top Drinks:** *Americano with Milk* and *Latte* dominate orders.  
- **Lowest Demand:** Espresso is the least ordered.  

---

### 3. Average Spending by Time of Day
- Customers spend around **30–32** on average across all times.  
- Spending is **slightly higher at night**, suggesting evening customers may prefer larger or premium drinks.  

---

### 4. Weekday vs Weekend Split
- **Average sales** are almost equal (**50.1% weekday vs 49.9% weekend**).  
- Weekdays: higher transaction counts.  
- Weekends: fewer customers but larger orders.  

---

### 5. Monthly & Seasonal Trends
- **Monthly:**  
  - *February & March* peak strongly, with **March 2025** having the highest sales.  
  - Mid-year (June–August) shows dips.  
- **Seasonal:**  
  - *Autumn (29%)* leads, followed by *Spring (26%)*.  
  - *Summer (20%)* is weakest.  

---

### 6. Coffee Type by Season
- **Autumn:** Americano with Milk is most popular.  
- **Spring:** Latte dominates.  
- **Summer:** Americano with Milk leads but overall demand is weaker.  
- **Winter:** Americano with Milk & Latte remain top sellers.  

---


